In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
result = llm.invoke("Who is Elong Musk?")
print(result.content)

Elon Musk is a business magnate, entrepreneur, and investor. He is best known for his ambitious goals and innovative ventures in the fields of electric cars, space exploration, renewable energy, and artificial intelligence.

Born on June 28, 1971, in Pretoria, South Africa, Musk moved to Canada in 1992 to attend college and later transferred to the University of Pennsylvania. After graduating, he moved to California to pursue a career in technology and entrepreneurship.

Musk's notable ventures include:

1. **PayPal**: In 1998, Musk co-founded PayPal, an online payment system that was later sold to eBay for $1.5 billion.
2. **SpaceX**: In 2002, Musk founded SpaceX, a private aerospace manufacturer and space transport services company, with the goal of reducing space transportation costs and enabling the colonization of Mars.
3. **Tesla, Inc.**: In 2004, Musk co-founded Tesla, Inc., which has become one of the leading electric vehicle manufacturers in the world.
4. **Neuralink**: In 201

In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
!pip install sentence_transformers

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


def main():
  print("Intializing Chatbot.........")
  llm = initialize_llm()

  db_path = "/content/chroma_db"

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm)

  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.run(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()


Intializing Chatbot.........

Human: Hello 
Chatbot: Hello. It's nice to meet you. I see you've been reading about mental health, specifically about how it's defined and the importance of considering the social and physical environments that influence our well-being. It's a fascinating topic, and I'm happy to discuss it with you.

The definitions you provided from Rowling et al. (2002) and the World Health Organization (2000, 2005b) highlight the importance of having a sense of purpose, quality relationships, self-respect, and mastery in achieving good mental health. However, as you noted, these definitions can be incomplete, as they don't fully account for the impact of our surroundings and the people around us on our mental health.

The additional definition you provided, which focuses on the capacity of individuals and groups to interact with one another and the environment in ways that promote subjective well-being, is a more comprehensive one. It acknowledges that mental health is

In [ ]:
!pip install gradio

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import gradio as gr
def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
  return llm

def create_vector_db():
  loader = DirectoryLoader("/content/data/", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever = vector_db.as_retriever()
  prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
  PROMPT = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

  qa_chain = RetrievalQA.from_chain_type(
      llm = llm,
      chain_type = "stuff",
      retriever = retriever,
      chain_type_kwargs = {"prompt": PROMPT}
  )
  return qa_chain


print("Intializing Chatbot.........")
llm = initialize_llm()

db_path = "/content/chroma_db"

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

import gradio as gr

# Replace this with your actual qa_chain logic
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "Please enter your message so I can assist you."
    response = qa_chain.run(user_input)  # <- Insert your real logic here
    return response

# Professional, soft aesthetic CSS
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;500;700&display=swap');

* {
    font-family: 'Inter', sans-serif;
}

/* Background: rich gradient */
.gradio-container {
    background: linear-gradient(135deg, #1f1c2c, #928dab) !important;
    min-height: 100vh;
    padding: 2rem;
    color: #f5f5f5 !important;
}

/* Chat interface container */
.gr-box {
    background-color: rgba(255, 255, 255, 0.07) !important;
    border-radius: 20px;
    backdrop-filter: blur(10px);
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.2);
    padding: 2rem;
}

/* Chatbot and messages */
.gr-chatbot, .gr-chatbot > div, .gr-chatbot .message {
    background-color: rgba(255, 255, 255, 0.05) !important;
    border-radius: 12px;
    padding: 10px;
}

/* Message bubbles */
.message.user, .message.bot {
    background-color: rgba(255, 255, 255, 0.15) !important;
    padding: 12px 16px;
    margin: 10px 0;
    border-radius: 14px;
    color: #ffffff !important;
}

/* Input styling */
textarea, input {
    background-color: rgba(255, 255, 255, 0.1) !important;
    color: #ffffff !important;
    border: 1px solid rgba(255, 255, 255, 0.3);
    border-radius: 12px;
}

/* For dark mode */
.dark .gr-box, .dark .gr-chatbot {
    background-color: rgba(0, 0, 0, 0.2) !important;
}
"""


# Launch the app
with gr.Blocks(css=custom_css) as app:
    gr.Markdown(
        """
        <h1 style='color:#D1C4E9; font-weight:700; text-align:center;'>🧘 Mental Wellness Companion 🤖</h1>
        <p style='text-align:center; font-size:1.1rem; max-width:700px; margin: auto; color: #e0e0e0;'>
            Welcome to your calm space for reflection and support. This chatbot offers a listening ear and gentle guidance.
            If you're facing a mental health crisis, please connect with a licensed professional or reach out to emergency services.
        </p>
        <br>
        """,
    )

    gr.ChatInterface(fn=chatbot_response, title="")

    gr.Markdown(
        "<p style='text-align:center; margin-top:2rem; color: #ccc;'>You're never alone — support is always within reach 💙</p>"
    )

app.launch()








Intializing Chatbot.........


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://71c8d9f9b3b75f19ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
